In [1]:
# clir_pipeline_b.py
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 1. Load corpus
tsv_path = '../fath_muin/cleaned_corpus.tsv'
df = pd.read_csv(tsv_path, sep="\t").dropna(subset=["text"])

2025-09-06 22:24:14.385362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-06 22:24:15.032581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757172255.354405    5982 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757172255.416316    5982 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757172255.829031    5982 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:

# 2. Load multilingual embedding model
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 3. Encode Arabic passages
doc_embeddings = model.encode(df["text"].tolist(), convert_to_tensor=True, show_progress_bar=True)

# 4. Example Indonesian query
query_id = "Apakah shalat itu wajib?"
query_embedding = model.encode(query_id, convert_to_tensor=True)

# 5. Compute cosine similarity
cos_scores = util.cos_sim(query_embedding, doc_embeddings)[0]

# 6. Rank results
top_k = 5
top_results = cos_scores.topk(top_k)

print("\n=== Top 5 Results (Pipeline B) ===")
for score, idx in zip(top_results.values, top_results.indices):
    print(f"[Score {score:.4f}] Doc {idx.item()}: {df.iloc[idx.item()]['text'][:200]}...")


Batches:   0%|          | 0/346 [00:00<?, ?it/s]


=== Top 5 Results (Pipeline B) ===
[Score 0.7079] Doc 80: بذلك لاشتمالها علي الصلاه لغه وهي الدعاء ....
[Score 0.7000] Doc 1445: وكذا دعاء محض ولو غير ماثور ....
[Score 0.6896] Doc 4811: صلاه : فيجب ركعتان بقيام قادر ....
[Score 0.6884] Doc 2442: فصل [ في صلاه الجماعه ]...
[Score 0.6881] Doc 2451: 650 ] : صلاه الجماعه افضل من صلاه الفذ بسبع وعشرين درجه ....


In [5]:
def query(query_id) :
    # 4. Example Indonesian query
    query_embedding = model.encode(query_id, convert_to_tensor=True)

    # 5. Compute cosine similarity
    cos_scores = util.cos_sim(query_embedding, doc_embeddings)[0]

    # 6. Rank results
    top_k = 5
    top_results = cos_scores.topk(top_k)

    print("\n=== Top 5 Results (Pipeline B) ===")
    for score, idx in zip(top_results.values, top_results.indices):
        print(f"[Score {score:.4f}] Doc {idx.item()}: {df.iloc[idx.item()]['text'][:400]}...")

query("pemilik membagi zakat")


=== Top 5 Results (Pipeline B) ===
[Score 0.7260] Doc 10161: الزكاه ....
[Score 0.7236] Doc 6169: رضا مالكه به ....
[Score 0.7067] Doc 3028: وستر للعوره ....
[Score 0.7006] Doc 2238: وزياد ....
[Score 0.6944] Doc 6699: المقر ....


In [6]:
df.head()

,line_id,text,tokens_space
0,1,| مقدمه,| مقدمه
1,2,. . .,. . .
2,3,بسم الله الرحمن الرحيم,بسم الله الرحمن الرحيم
3,4,الحمد لله الفتاح الجواد المعين,الحمد لله الفتاح الجواد المعين
4,5,علي التفقه في الدين من اختاره من العباد واشهد ...,علي التفقه في الدين من اختاره من العباد واشهد ...
